In [2]:
from tensorflow.keras.models import load_model
from keras import backend as K
from tensorflow.keras.utils import CustomObjectScope
from tensorflow.keras.initializers import glorot_uniform

import pickle
import numpy as np
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

d:\programfiles\anaconda\envs\nlp\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
d:\programfiles\anaconda\envs\nlp\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
d:\programfiles\anaconda\envs\nlp\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
d:\programfiles\anaconda\envs\nlp\lib\site-packages\tensorflow\python\framework

True

In [9]:
with open('./web/curse-words-Google.txt') as curseWordFile:
    curseWordList = curseWordFile.read().split('\n')
curseWordDict = {}
for curseWordIndex in range(len(curseWordList)):
    curseWordDict[curseWordList[curseWordIndex]] = curseWordIndex

    
def IsSentenceCursed(sentence):
    for item in inputSentence.split():
        if(curseWordDict.get(item)):
            return True
    return False

In [10]:
def filterStopWords(sentence):
    stop_words = set(stopwords.words('english')) 
    filteredSentence = " ".join([w for w in sentence.split(' ') if not w in stop_words])
    return filteredSentence

In [5]:
with open('./backup/wordVectorizer/vectorizer.pkl', 'rb') as vectorizerFile:
    vectorizer = pickle.load(vectorizerFile)

In [6]:
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [7]:
# Loading the model

model = load_model('./backup/model/baselineMLP/zs_mlp_model.h5', custom_objects= {'f1': f1}) # We serialized the weights of the feed-forward network

W0915 19:35:11.361519  5432 nn_ops.py:4220] Large dropout rate: 0.7 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.
W0915 19:35:14.672738  5432 nn_ops.py:4220] Large dropout rate: 0.7 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.
W0915 19:35:14.772706  5432 nn_ops.py:4220] Large dropout rate: 0.7 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.
W0915 19:35:15.325424  5432 deprecation.py:323] From d:\programfiles\anaconda\envs\nlp\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [15]:
inputSentence = 'join my institute for better understanding on such topics at www.mltraining.com'

# Checking for curse words contain in sentence
if IsSentenceCursed(inputSentence) == False:
    
    # Filtering stop words from sentence
    filteredSentence = filterStopWords(inputSentence)
    
    # Vectorizing sentence for model input
    inputData = vectorizer.transform(np.array([filteredSentence]))
    
    if model.predict_classes(inputData)[0][0] == 1:
        print('Identified spam comment')
    else:
        print('Identified non-spam comment')

join institute better understanding topics www.mltraining.com
Identified spam comment
